In [1]:
import sqlalchemy

In [2]:
sqlalchemy.__version__

'1.3.13'

In [3]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=True)

In [4]:
engine

Engine(sqlite:///:memory:)

In [5]:
from sqlalchemy import MetaData, Table, Column, Integer, String, ForeignKey

meta = MetaData()
users = Table('users', meta,
              Column('id', Integer, primary_key=True),
              Column('name', String),
              Column('fullname', String))

addr = Table('address', meta,
             Column('id', Integer, primary_key=True),
             Column('userid', Integer, ForeignKey('users.id')),
             Column('email', String, nullable=True))

In [6]:
meta.create_all(engine)

2020-09-22 14:19:27,018 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-09-22 14:19:27,027 INFO sqlalchemy.engine.base.Engine ()
2020-09-22 14:19:27,030 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-09-22 14:19:27,031 INFO sqlalchemy.engine.base.Engine ()
2020-09-22 14:19:27,034 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("users")
2020-09-22 14:19:27,036 INFO sqlalchemy.engine.base.Engine ()
2020-09-22 14:19:27,038 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("users")
2020-09-22 14:19:27,041 INFO sqlalchemy.engine.base.Engine ()
2020-09-22 14:19:27,045 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("address")
2020-09-22 14:19:27,046 INFO sqlalchemy.engine.base.Engine ()
2020-09-22 14:19:27,050 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("address")
2020-09-22 14:19:27,052 INFO sqlalchemy.engine.base.Engine ()
2020-09-22 14:19:27,05

In [7]:
meta.tables

immutabledict({'users': Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), schema=None), 'address': Table('address', MetaData(bind=None), Column('id', Integer(), table=<address>, primary_key=True, nullable=False), Column('userid', Integer(), ForeignKey('users.id'), table=<address>), Column('email', String(), table=<address>), schema=None)})

In [8]:
meta.bind # 아직은 바인딩 되지 않음 # db랑 상관없다는 의미

In [9]:
print(users.insert())

INSERT INTO users (id, name, fullname) VALUES (:id, :name, :fullname)


In [10]:
insert = users.insert().values(name='test', fullname='testtest')
print(insert)
print(insert.compile())
print(insert.compile().params)

INSERT INTO users (name, fullname) VALUES (:name, :fullname)
INSERT INTO users (name, fullname) VALUES (:name, :fullname)
{'name': 'test', 'fullname': 'testtest'}


In [11]:
conn = engine.connect()

In [12]:
conn.execute(insert)

2020-09-22 14:19:30,744 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2020-09-22 14:19:30,747 INFO sqlalchemy.engine.base.Engine ('test', 'testtest')
2020-09-22 14:19:30,750 INFO sqlalchemy.engine.base.Engine COMMIT


In [13]:
print(users.select().compile())

SELECT users.id, users.name, users.fullname 
FROM users


In [14]:
conn.execute(users.select())

2020-09-22 14:19:31,648 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2020-09-22 14:19:31,650 INFO sqlalchemy.engine.base.Engine ()


In [15]:
cur = conn.execute(users.select())

2020-09-22 14:19:32,088 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2020-09-22 14:19:32,097 INFO sqlalchemy.engine.base.Engine ()


In [16]:
list(cur)

[(1, 'test', 'testtest')]

In [17]:
cur = conn.execute(users.insert(), {'name':'test2', 'fullname':'test2test2'})

2020-09-22 14:19:33,841 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2020-09-22 14:19:33,843 INFO sqlalchemy.engine.base.Engine ('test2', 'test2test2')
2020-09-22 14:19:33,852 INFO sqlalchemy.engine.base.Engine COMMIT


In [18]:
cur.lastrowid

2

In [19]:
cur = conn.execute(users.select())

2020-09-22 14:19:35,881 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2020-09-22 14:19:35,883 INFO sqlalchemy.engine.base.Engine ()


In [20]:
print(users.c.id == 1)
print((users.c.id == 1).compile().params)

users.id = :id_1
{'id_1': 1}


In [21]:
cur = conn.execute(users.select().where(users.c.id==1))
list(cur)

2020-09-22 14:19:39,578 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users 
WHERE users.id = ?
2020-09-22 14:19:39,580 INFO sqlalchemy.engine.base.Engine (1,)


[(1, 'test', 'testtest')]

In [22]:
cur = conn.execute(addr.insert(), {'usersid':1, 'email':'1@1.com'})

2020-09-22 14:19:40,551 INFO sqlalchemy.engine.base.Engine INSERT INTO address (email) VALUES (?)
2020-09-22 14:19:40,554 INFO sqlalchemy.engine.base.Engine ('1@1.com',)
2020-09-22 14:19:40,558 INFO sqlalchemy.engine.base.Engine COMMIT


In [23]:
cur.lastrowid

1